### Imports

In [18]:
import os
import cv2
import pickle
from pytesseract import Output, image_to_data

### Tesseract Predictions for Docbank data without Images

In [19]:
input_dir   = './docbank_processed/processed_data/ori_black/'
output_dir  = './docbank_processed/resultant_data/tesseract/' 
dir_list = os.listdir(input_dir)

In [20]:
dimensions = {}
results = {}
total = {}

for image in dir_list:
    predictions = []
    img = cv2.imread(input_dir + image)
    d = image_to_data(img, output_type=Output.DICT)
    for i in range(len(d['level'])):
        values = []
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        (x, y, w, h) = (int(x), int(y), int(w), int(h))
        values.append('text')
        values.append(d['conf'][i])
        values = values + [x,y,w,h]
        values.append(d['text'][i])
        if(x==0 and y==0):
            continue
        else:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255))
        predictions.append(values)
    total[image] = predictions
    results[image] = d
    cv2.imwrite(output_dir + 'images/' + image, img)

### Store Results

In [21]:
for image,result in total.items():
    name = image[:len(image) - 12]
    df = pd.DataFrame(result, columns = ['class', 'confidence', 'X', 'Y', 'W', 'H', 'prediction'])
    df.to_csv(output_dir + 'txt/' + name + '.txt', sep='\t',index=False)

NameError: name 'pd' is not defined

In [ ]:
with open(output_dir+'results.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)